In [96]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode
import os
from nltk.tokenize import word_tokenize

from statistics import mode
print("done")

done


In [97]:
short_neg=open("negative.txt","r").read()
short_pos=open("positive.txt","r").read()
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
print("done")

done


In [146]:
documents = []


for r in short_pos.split('\n'):
    documents.append( (r, "pos") )

for r in short_neg.split('\n'):
    documents.append( (r, "neg") )

all_words=[]
short_pos_words=word_tokenize(short_pos)
short_neg_words=word_tokenize(short_neg)
for w in short_pos_words:
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features=list(all_words.keys())[:5000]
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features
print("1")
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)
print("2")
# positive data example:      
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]

##
### negative data example:      
##training_set = featuresets[100:]
##testing_set =  featuresets[:100]


classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)
print("trained 1")
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
print("trained 2")
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)
print("trained 3")
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)
print("trained 4")
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)
print("trained 5")
##SVC_classifier = SklearnClassifier(SVC())
##SVC_classifier.train(training_set)
##print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)
print("trained 6")
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)
print("trained 7")

voted_classifier = VoteClassifier(
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)


1
2
Naive Bayes Algo accuracy percent: 72.13855421686746
Most Informative Features
              engrossing = True              pos : neg    =     21.7 : 1.0
               inventive = True              pos : neg    =     15.0 : 1.0
              refreshing = True              pos : neg    =     14.4 : 1.0
               wonderful = True              pos : neg    =     13.0 : 1.0
            refreshingly = True              pos : neg    =     13.0 : 1.0
                    warm = True              pos : neg    =     12.6 : 1.0
               realistic = True              pos : neg    =     11.7 : 1.0
                captures = True              pos : neg    =     11.0 : 1.0
                provides = True              pos : neg    =     10.6 : 1.0
                  stupid = True              neg : pos    =     10.6 : 1.0
                 quietly = True              pos : neg    =     10.4 : 1.0
             mesmerizing = True              pos : neg    =     10.4 : 1.0
                 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 71.53614457831326
trained 5
LinearSVC_classifier accuracy percent: 73.04216867469879
trained 6
NuSVC_classifier accuracy percent: 74.54819277108435
trained 7
voted_classifier accuracy percent: 74.54819277108435


In [150]:
# run to pickle everything

save_documents=open("pickled_algos/document.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

save_word_features=open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

#print(len(MNB_classifier))
save_classifier=open("pickled_algos/MNB_classifier5k.pickle","wb")
print("pickle of save_MNB:",save_MNB_classifier)
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

print(classifier)
save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDClassifier_classifier, save_classifier)
save_classifier.close()

print("all pickled up!")

pickle of save_MNB: <_io.BufferedWriter name='pickled_algos/MNB_classifier5k.pickle'>
all pickled up!


In [153]:
open_file=open("pickled_algos/document.pickle", "rb")
documents=pickle.load(open_file)
open_file.close()


open_file = open("pickled_algos/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("pickled_algos/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("pickled_algos/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("pickled_algos/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("pickled_algos/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("pickled_algos/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()

#open_file=open("pickled_algos/word_features5k.pickle","wb")
#word_features=pickle.load(open_file)
#open_file.close()

#print("IT WORK!")

In [196]:
def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)

print(sentiment("Thor Ragnarok was a great movie. It was good. Really good. amazing. Wonderful. Engrossing!"))
print(sentiment("this movie sucked. I was getting bored all the time. The movie was too long."))
print(sentiment("This movie had some great acting, but the storyline was lacking. Overall, I was not impressed."))

('pos', 0.8)
('neg', 1.0)
('neg', 1.0)


In [ ]:
"""# analysis of tweets about movies
from tweepy import Stream
from tweepy import OAuthHandler as OAH
from tweepy.streaming import streamListener

#keys
ckey=
csecret=
atoken=
asecret=

class listener(StreamListener):
    def on_data(self,data):
        print(data)
        return True
    def on_error(self,status):
        print status
auth=OAH(ckey, csecret)
auth.set_access_token(atoken, asecret)
twitterStream=Stream(auth,listener())
twitterStream.filter(track=[""])
"""
